In [1]:
already_loaded = True
augmented = False
oversampled = True
transformed = True
trained_mlp = True
trained_cnn_125x94 = True
resized_32x32 = True
trained_cnn_resized = True
extracted_hog = True
hog_pca_transformed = True
trained_hog_svm = True
extracted_lbp = True
lbp_pca_transformed = True
trained_lbp_svm = True

#### **Imports**

In [177]:
from joblib import dump, load
import numpy as np
from collections import Counter
import os
import cv2
import random

from sklearn.utils import shuffle
from sklearn.datasets import fetch_lfw_people

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

from skimage.transform import resize
from skimage.feature import hog
from skimage.feature import local_binary_pattern

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms

from PIL import Image

from imblearn.over_sampling import SMOTE, ADASYN

import matplotlib.pyplot as plt
import seaborn as sns

#### **Loading Dataset**

In [105]:
if not already_loaded:
    
    # Load the LFW dataset in color with original image size
    lfw_people = fetch_lfw_people(color=True, resize=None, min_faces_per_person=20)

    # fetch grayscale iamges
    lfw_people_gray = fetch_lfw_people(color=False, resize=None, min_faces_per_person=20)
    X_gray = lfw_people_gray.data
    y_gray = lfw_people_gray.target
    dump(lfw_people_gray, 'lfw_dataset_gray.joblib')

    n_samples, h, w, c = lfw_people.images.shape

    X = lfw_people.data
    n_features = X.shape[1]

    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]

    print("Total dataset size:")
    print("n_samples: %d" % n_samples)
    print(f"Image dimensions: {h} x {w} x {c}")
    print("n_features: %d" % n_features)
    print("n_classes: %d" % n_classes)

    dump(lfw_people, 'lfw_dataset.joblib')

else:
    # Load the dataset from the saved file
    lfw_people = load('lfw_dataset.joblib')

    lfw_people_gray = load('lfw_dataset_gray.joblib')
    X_gray = lfw_people_gray.data
    y_gray = lfw_people_gray.target

    n_samples, h, w, c = lfw_people.images.shape

    X = lfw_people.data
    n_features = X.shape[1]

    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]

    print("Total dataset size:")
    print("n_samples: %d" % n_samples)
    print(f"Image dimensions: {h} x {w} x {c}")
    print("n_features: %d" % n_features)
    print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 2489
Image dimensions: 125 x 94 x 3
n_features: 35250
n_classes: 43


In [60]:
if not resized_32x32:

    # Load the LFW dataset in color with original image size
    lfw_people_orig = fetch_lfw_people(color=True, resize=0.35, min_faces_per_person=20)

    n_samples_orig, h_orig, w_orig, c_orig = lfw_people_orig.images.shape

    X_orig = lfw_people_orig.data
    n_features_orig = X_orig.shape[1]

    y_orig = lfw_people_orig.target
    target_names_orig = lfw_people_orig.target_names
    n_classes_orig = target_names_orig.shape[0]

    # Reshape images
    X_reshaped = lfw_people_orig.images.reshape(n_samples_orig, h_orig, w_orig, c_orig)

    # Crop images
    crop_top = int((h_orig - 32) / 2)
    crop_bottom = h_orig - 32 - crop_top
    lfw_people_cropped = X_reshaped[:, crop_top:-crop_bottom, :, :]

    # Reshape again after cropping
    n_samples, h_resized, w_resized, c = lfw_people_cropped.shape
    X_resized = lfw_people_cropped.reshape(n_samples, -1)
    n_features_resized = X_resized.shape[1]
    y = y_orig
    target_names = target_names_orig
    n_classes = n_classes_orig

    print("Total dataset size:")
    print("n_samples: %d" % n_samples)
    print(f"Image dimensions: {h_resized} x {w_resized} x {c}")
    print("n_features: %d" % n_features_resized)
    print("n_classes: %d" % n_classes)

    dump(lfw_people_cropped, 'lfw_dataset_resized_cropped.joblib')

else:
    # Load the dataset from the saved file
    lfw_people_cropped = load('lfw_dataset_resized_cropped.joblib')

    n_samples, h_resized, w_resized, c = lfw_people_cropped.shape

    X_resized = lfw_people_cropped.reshape(n_samples, -1)
    n_features_resized = X_resized.shape[1]

    y = lfw_people.target
    target_names = lfw_people.target_names
    n_classes = target_names.shape[0]

    print("Total dataset size:")
    print("n_samples: %d" % n_samples)
    print(f"Image dimensions: {h_resized} x {w_resized} x {c}")
    print("n_features: %d" % n_features_resized)
    print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 2489
Image dimensions: 32 x 32 x 3
n_features: 3072
n_classes: 43


#### **Train Test Split**

In [61]:
# Split data into train, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.125, random_state=42, stratify=y_train_val)

In [106]:
# Split data into train, validation, and test sets
X_train_val_gray, X_test_gray, y_train_val_gray, y_test_gray = train_test_split(X_gray, y_gray, test_size=0.2, random_state=42, stratify=y_gray)
X_train_gray, X_val_gray, y_train_gray, y_val_gray = train_test_split(X_train_val_gray, y_train_val_gray, test_size=0.125, random_state=42, stratify=y_train_val_gray)

In [62]:
# Split data into train, validation, and test sets
X_train_val_resized, X_test_resized, y_train_val, y_test = train_test_split(X_resized, y, test_size=0.2, random_state=42, stratify=y)
X_train_resized, X_val_resized, y_train, y_val = train_test_split(X_train_val_resized, y_train_val, test_size=0.125, random_state=42, stratify=y_train_val)

#### **SMOTE & ADASYN oversampling**

In [107]:
if not oversampled:

    # Apply SMOTE oversampling only on the training set
    smote = SMOTE(random_state=42)
    X_train_resampled_smote, y_train_resampled_smote = smote.fit_resample(X_train, y_train)

    # Count the class distribution after SMOTE oversampling
    print("Class distribution after SMOTE oversampling:", np.bincount(y_train_resampled_smote))


    smote_gray = SMOTE(random_state = 42)
    X_train_gray_resampled_smote, y_train_gray_resampled_smote = smote_gray.fit_resample(X_train_gray, y_train_gray)

    # Count the class distribution after SMOTE oversampling
    print("Class distribution after SMOTE oversampling on grayscale:", np.bincount(y_train_gray_resampled_smote))


    smote_resized = SMOTE(random_state=42)
    X_train_resampled_smote_resized, y_train_resampled_smote_resized = smote_resized.fit_resample(X_train_resized, y_train)

    # Count the class distribution after SMOTE oversampling
    print("Class distribution after SMOTE oversampling on resized:", np.bincount(y_train_resampled_smote_resized))


    # Apply ADASYN oversampling only on the training set
    adasyn = ADASYN(random_state=42)
    X_train_resampled_adasyn, y_train_resampled_adasyn = adasyn.fit_resample(X_train, y_train)

    # Count the class distribution after ADASYN oversampling
    print("Class distribution after ADASYN oversampling:", np.bincount(y_train_resampled_adasyn))


    # Apply ADASYN oversampling only on the training set
    adasyn_gray = ADASYN(random_state=42)
    X_train_gray_resampled_adasyn, y_train_gray_resampled_adasyn = adasyn_gray.fit_resample(X_train_gray, y_train_gray)

    # Count the class distribution after ADASYN oversampling
    print("Class distribution after ADASYN oversampling on grayscale:", np.bincount(y_train_gray_resampled_adasyn))


    adasyn_resized = ADASYN(random_state=42)
    X_train_resampled_adasyn_resized, y_train_resampled_adasyn_resized = adasyn_resized.fit_resample(X_train_resized, y_train)

    # Count the class distribution after SMOTE oversampling
    print("Class distribution after ADASYN oversampling on resized:", np.bincount(y_train_resampled_adasyn_resized))


    dump(X_train_resampled_smote, 'X_train_resampled_smote.joblib')
    dump(y_train_resampled_smote, 'y_train_resampled_smote.joblib')

    dump(X_train_resampled_adasyn, 'X_train_resampled_adasyn.joblib')
    dump(y_train_resampled_adasyn, 'y_train_resampled_adasyn.joblib')

    dump(X_train_gray_resampled_smote, 'X_train_gray_resampled_smote.joblib')
    dump(y_train_gray_resampled_smote, 'y_train_gray_resampled_smote.joblib')

    dump(X_train_gray_resampled_adasyn, 'X_train_gray_resampled_adasyn.joblib')
    dump(y_train_gray_resampled_adasyn, 'y_train_gray_resampled_adasyn.joblib')

    dump(X_train_resampled_smote_resized, 'X_train_resampled_smote_resized.joblib')
    dump(y_train_resampled_smote_resized, 'y_train_resampled_smote_resized.joblib')

    dump(X_train_resampled_adasyn_resized, 'X_train_resampled_adasyn_resized.joblib')
    dump(y_train_resampled_adasyn_resized, 'y_train_resampled_adasyn_resized.joblib')

else:

    X_train_resampled_smote = load('X_train_resampled_smote.joblib')
    y_train_resampled_smote = load('y_train_resampled_smote.joblib')

    X_train_resampled_adasyn = load('X_train_resampled_adasyn.joblib')
    y_train_resampled_adasyn = load('y_train_resampled_adasyn.joblib')

    X_train_gray_resampled_smote = load('X_train_gray_resampled_smote.joblib')
    y_train_gray_resampled_smote = load('y_train_gray_resampled_smote.joblib')

    X_train_gray_resampled_adasyn = load('X_train_gray_resampled_adasyn.joblib')
    y_train_gray_resampled_adasyn = load('y_train_gray_resampled_adasyn.joblib')

    X_train_resampled_smote_resized = load('X_train_resampled_smote_resized.joblib')
    y_train_resampled_smote_resized = load('y_train_resampled_smote_resized.joblib')

    X_train_resampled_adasyn_resized = load('X_train_resampled_adasyn_resized.joblib')
    y_train_resampled_adasyn_resized = load('y_train_resampled_adasyn_resized.joblib')

Class distribution after SMOTE oversampling on grayscale: [371 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371
 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371 371
 371 371 371 371 371 371 371]
Class distribution after ADASYN oversampling on grayscale: [366 360 370 367 362 369 364 383 370 362 371 371 399 367 376 364 368 367
 372 367 358 365 375 376 373 379 366 373 372 366 380 373 374 369 370 374
 372 367 371 379 368 367 378]


#### **PCA & LDA Transform**

In [108]:
if not transformed:

    # Perform PCA after oversampling
    pca_smote = PCA(n_components=100)
    pca_adasyn = PCA(n_components=100)

    pca_smote_gray = PCA(n_components=100)
    pca_adasyn_gray = PCA(n_components=100)

    pca_smote_resized = PCA(n_components=100)
    pca_adasyn_resized = PCA(n_components=100)


    X_train_pca_smote = pca_smote.fit_transform(X_train_resampled_smote)
    X_train_pca_adasyn = pca_adasyn.fit_transform(X_train_resampled_adasyn)

    X_val_pca_smote = pca_smote.transform(X_val)
    X_test_pca_smote = pca_smote.transform(X_test)

    X_val_pca_adasyn = pca_adasyn.transform(X_val)
    X_test_pca_adasyn = pca_adasyn.transform(X_test)


    X_train_gray_pca_smote = pca_smote_gray.fit_transform(X_train_gray_resampled_smote)
    X_train_gray_pca_adasyn = pca_adasyn_gray.fit_transform(X_train_gray_resampled_adasyn)

    X_val_gray_pca_smote = pca_smote_gray.transform(X_val_gray)
    X_test_gray_pca_smote = pca_smote_gray.transform(X_test_gray)

    X_val_gray_pca_adasyn = pca_adasyn_gray.transform(X_val_gray)
    X_test_gray_pca_adasyn = pca_adasyn_gray.transform(X_test_gray)


    X_train_pca_smote_resized = pca_smote_resized.fit_transform(X_train_resampled_smote_resized)
    X_train_pca_adasyn_resized = pca_adasyn_resized.fit_transform(X_train_resampled_adasyn_resized)

    X_val_pca_smote_resized = pca_smote_resized.transform(X_val_resized)
    X_test_pca_smote_resized = pca_smote_resized.transform(X_test_resized)

    X_val_pca_adasyn_resized = pca_adasyn_resized.transform(X_val_resized)
    X_test_pca_adasyn_resized = pca_adasyn_resized.transform(X_test_resized)


    # Save the transformed data
    dump(X_train_pca_smote, 'X_train_pca_smote.joblib')
    dump(X_val_pca_smote, 'X_val_pca_smote.joblib')
    dump(X_test_pca_smote, 'X_test_pca_smote.joblib')

    dump(X_train_pca_adasyn, 'X_train_pca_adasyn.joblib')
    dump(X_val_pca_adasyn, 'X_val_pca_adasyn.joblib')
    dump(X_test_pca_adasyn, 'X_test_pca_adasyn.joblib')


    dump(X_train_gray_pca_smote, 'X_train_gray_pca_smote.joblib')
    dump(X_val_gray_pca_smote, 'X_val_gray_pca_smote.joblib')
    dump(X_test_gray_pca_smote, 'X_test_gray_pca_smote.joblib')

    dump(X_train_gray_pca_adasyn, 'X_train_gray_pca_adasyn.joblib')
    dump(X_val_gray_pca_adasyn, 'X_val_gray_pca_adasyn.joblib')
    dump(X_test_gray_pca_adasyn, 'X_test_gray_pca_adasyn.joblib')


    dump(X_train_pca_smote_resized, 'X_train_pca_smote_resized.joblib')
    dump(X_val_pca_smote_resized, 'X_val_pca_smote_resized.joblib')
    dump(X_test_pca_smote_resized, 'X_test_pca_smote_resized.joblib')

    dump(X_train_pca_adasyn_resized, 'X_train_pca_adasyn_resized.joblib')
    dump(X_val_pca_adasyn_resized, 'X_val_pca_adasyn_resized.joblib')
    dump(X_test_pca_adasyn_resized, 'X_test_pca_adasyn_resized.joblib')

    # Perform LDA after oversampling
    # lda_smote = LDA(n_components=None)
    # lda_adasyn = LDA(n_components=None)

    # X_train_lda_smote = lda_smote.fit_transform(X_train_resampled_smote, y_train_resampled_smote)
    # X_train_lda_adasyn = lda_adasyn.fit_transform(X_train_resampled_adasyn, y_train_resampled_adasyn)

    # X_val_lda_smote = lda_smote.transform(X_val)
    # X_test_lda_smote = lda_smote.transform(X_test)

    # X_val_lda_adasyn = lda_adasyn.transform(X_val)
    # X_test_lda_adasyn = lda_adasyn.transform(X_test)

    # dump(X_train_lda_smote, 'X_train_lda_smote.joblib')
    # dump(X_val_lda_smote, 'X_val_lda_smote.joblib')
    # dump(X_test_lda_smote, 'X_test_lda_smote.joblib')

    # dump(X_train_lda_adasyn, 'X_train_lda_adasyn.joblib')
    # dump(X_val_lda_adasyn, 'X_val_lda_adasyn.joblib')
    # dump(X_test_lda_adasyn, 'X_test_lda_adasyn.joblib')

else:

    # Save the transformed data
    X_train_pca_smote = load('X_train_pca_smote.joblib')
    X_val_pca_smote = load('X_val_pca_smote.joblib')
    X_test_pca_smote = load('X_test_pca_smote.joblib')

    X_train_gray_pca_smote = load('X_train_gray_pca_smote.joblib')
    X_val_gray_pca_smote = load('X_val_gray_pca_smote.joblib')
    X_test_gray_pca_smote = load('X_test_gray_pca_smote.joblib')

    X_train_pca_smote_resized = load('X_train_pca_smote_resized.joblib')
    X_val_pca_smote_resized = load('X_val_pca_smote_resized.joblib')
    X_test_pca_smote_resized = load('X_test_pca_smote_resized.joblib')

    # X_train_lda_smote = load('X_train_lda_smote.joblib')
    # X_val_lda_smote = load('X_val_lda_smote.joblib')
    # X_test_lda_smote = load('X_test_lda_smote.joblib')

    X_train_pca_adasyn = load('X_train_pca_adasyn.joblib')
    X_val_pca_adasyn = load('X_val_pca_adasyn.joblib')
    X_test_pca_adasyn = load('X_test_pca_adasyn.joblib')

    X_train_gray_pca_adasyn = load('X_train_gray_pca_adasyn.joblib')
    X_val_gray_pca_adasyn = load('X_val_gray_pca_adasyn.joblib')
    X_test_gray_pca_adasyn = load('X_test_gray_pca_adasyn.joblib')

    X_train_pca_adasyn_resized = load('X_train_pca_adasyn_resized.joblib')
    X_val_pca_adasyn_resized = load('X_val_pca_adasyn_resized.joblib')
    X_test_pca_adasyn_resized = load('X_test_pca_adasyn_resized.joblib')

    # X_train_lda_adasyn = load('X_train_lda_adasyn.joblib')
    # X_val_lda_adasyn = load('X_val_lda_adasyn.joblib')
    # X_test_lda_adasyn = load('X_test_lda_adasyn.joblib')

#### **Device, tensors & Dataloaders**

In [110]:
# Check if GPU is available
# device = torch.device("cuda")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


##### **For MLP**

In [66]:
X_train_tensor_pca = torch.tensor(X_train_pca_smote, dtype=torch.float32).to(device)
y_train_tensor_pca = torch.tensor(y_train_resampled_smote, dtype=torch.long).to(device)
X_val_tensor_pca = torch.tensor(X_val_pca_smote, dtype=torch.float32).to(device)
y_val_tensor_pca = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor_pca = torch.tensor(X_test_pca_smote, dtype=torch.float32).to(device)
y_test_tensor_pca = torch.tensor(y_test, dtype=torch.long).to(device)

In [67]:
train_dataset_pca = TensorDataset(X_train_tensor_pca, y_train_tensor_pca)
train_loader_pca = DataLoader(train_dataset_pca, batch_size=64, shuffle=True)

val_dataset_pca = TensorDataset(X_val_tensor_pca, y_val_tensor_pca)
val_loader_pca = DataLoader(val_dataset_pca, batch_size=64, shuffle=True)

test_dataset_pca = TensorDataset(X_test_tensor_pca, y_test_tensor_pca)
test_loader_pca = DataLoader(test_dataset_pca, batch_size=64, shuffle=True)

In [68]:
X_train_tensor_pca_resized = torch.tensor(X_train_pca_smote_resized, dtype=torch.float32).to(device)
y_train_tensor_pca_resized = torch.tensor(y_train_resampled_smote_resized, dtype=torch.long).to(device)
X_val_tensor_pca_resized = torch.tensor(X_val_pca_smote_resized, dtype=torch.float32).to(device)
y_val_tensor_pca_resized = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor_pca_resized = torch.tensor(X_test_pca_smote_resized, dtype=torch.float32).to(device)
y_test_tensor_pca_resized = torch.tensor(y_test, dtype=torch.long).to(device)

In [69]:
train_dataset_pca_resized = TensorDataset(X_train_tensor_pca_resized, y_train_tensor_pca_resized)
train_loader_pca_resized = DataLoader(train_dataset_pca_resized, batch_size=64, shuffle=True)

val_dataset_pca_resized = TensorDataset(X_val_tensor_pca_resized, y_val_tensor_pca_resized)
val_loader_pca = DataLoader(val_dataset_pca, batch_size=64, shuffle=True)

test_dataset_pca_resized = TensorDataset(X_test_tensor_pca_resized, y_test_tensor_pca_resized)
test_loader_pca_resized = DataLoader(test_dataset_pca_resized, batch_size=64, shuffle=True)

##### **For CNN**

In [70]:
X_train_resampled_smote_3d = X_train_resampled_smote.reshape(-1, 3, 125, 94)
X_val_3d = X_val.reshape(-1, 3, 125, 94)
X_test_3d = X_test.reshape(-1, 3, 125, 94)

In [111]:
X_train_gray_resampled_smote_2d = X_train_gray_resampled_smote.reshape(-1, 125, 94)
X_val_gray_2d = X_val_gray.reshape(-1, 125, 94)
X_test_gray_2d = X_test_gray.reshape(-1, 125, 94)

In [72]:
X_train_resampled_smote_3d_resized = X_train_resampled_smote_resized.reshape(-1, 3, 32, 32)
X_val_3d_resized = X_val_resized.reshape(-1, 3, 32, 32)
X_test_3d_resized = X_test_resized.reshape(-1, 3, 32, 32)

##### **Flattened**

In [73]:
X_train_tensor = torch.tensor(X_train_resampled_smote, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_resampled_smote, dtype=torch.long).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [74]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [75]:
X_train_tensor_resized = torch.tensor(X_train_resampled_smote_resized, dtype=torch.float32).to(device)
y_train_tensor_resized = torch.tensor(y_train_resampled_smote_resized, dtype=torch.long).to(device)
X_val_tensor_resized = torch.tensor(X_val_resized, dtype=torch.float32).to(device)
y_val_tensor_resized = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor_resized = torch.tensor(X_test_resized, dtype=torch.float32).to(device)
y_test_tensor_resized = torch.tensor(y_test, dtype=torch.long).to(device)

In [76]:
train_dataset_resized = TensorDataset(X_train_tensor_resized, y_train_tensor_resized)
train_loader_resized = DataLoader(train_dataset_resized, batch_size=64, shuffle=True)

val_dataset_resized = TensorDataset(X_val_tensor_resized, y_val_tensor_resized)
val_loader_resized = DataLoader(val_dataset_resized, batch_size=64, shuffle=True)

test_dataset_resized = TensorDataset(X_test_tensor_resized, y_test_tensor_resized)
test_loader_resized = DataLoader(test_dataset_resized, batch_size=64, shuffle=True)

##### **3D**

In [77]:
X_train_tensor_3d = torch.tensor(X_train_resampled_smote_3d, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_resampled_smote, dtype=torch.long).to(device)
X_val_tensor_3d = torch.tensor(X_val_3d, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor_3d = torch.tensor(X_test_3d, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

In [78]:
train_dataset_3d = TensorDataset(X_train_tensor_3d, y_train_tensor)
train_loader_3d = DataLoader(train_dataset_3d, batch_size=64, shuffle=True)

val_dataset_3d = TensorDataset(X_val_tensor_3d, y_val_tensor)
val_loader_3d = DataLoader(val_dataset_3d, batch_size=64, shuffle=True)

test_dataset_3d = TensorDataset(X_test_tensor_3d, y_test_tensor)
test_loader_3d = DataLoader(test_dataset_3d, batch_size=64, shuffle=True)

In [79]:
X_train_tensor_3d_resized = torch.tensor(X_train_resampled_smote_3d_resized, dtype=torch.float32).to(device)
y_train_tensor_resized = torch.tensor(y_train_resampled_smote_resized, dtype=torch.long).to(device)
X_val_tensor_3d_resized = torch.tensor(X_val_3d_resized, dtype=torch.float32).to(device)
y_val_tensor_resized = torch.tensor(y_val, dtype=torch.long).to(device)
X_test_tensor_3d_resized = torch.tensor(X_test_3d_resized, dtype=torch.float32).to(device)
y_test_tensor_resized = torch.tensor(y_test, dtype=torch.long).to(device)

In [80]:
train_dataset_3d_resized = TensorDataset(X_train_tensor_3d_resized, y_train_tensor_resized)
train_loader_3d_resized = DataLoader(train_dataset_3d_resized, batch_size=64, shuffle=True)

val_dataset_3d_resized = TensorDataset(X_val_tensor_3d_resized, y_val_tensor_resized)
val_loader_3d_resized = DataLoader(val_dataset_3d_resized, batch_size=64, shuffle=True)

test_dataset_3d_resized = TensorDataset(X_test_tensor_3d_resized, y_test_tensor_resized)
test_loader_3d_resized = DataLoader(test_dataset_3d_resized, batch_size=64, shuffle=True)

#### **MLP**

In [81]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model_mlp(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            model = model.to(device)
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

def eval_mlp_model(mlp_model, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor):

    mlp_model.eval()
    with torch.no_grad():
        train_outputs = mlp_model(X_train_tensor)
        _, train_predicted = torch.max(train_outputs, 1)
        train_accuracy = accuracy_score(y_train_tensor.cpu().numpy(), train_predicted.cpu().numpy())
        print("Train Accuracy:", train_accuracy)

        val_outputs = mlp_model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs, 1)
        val_accuracy = accuracy_score(y_val_tensor.cpu().numpy(), val_predicted.cpu().numpy())
        print("Validation Accuracy:", val_accuracy)

In [82]:
input_size = X_train_tensor_pca.shape[1]
hidden_size = 128
output_size = len(torch.unique(y_train_tensor_pca))
mlp_model_pca = MLPClassifier(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model_pca.parameters(), lr=0.001)

In [83]:
if not trained_mlp:

    train_model_mlp(mlp_model_pca, train_loader_pca, criterion, optimizer, num_epochs=10)

    eval_mlp_model(mlp_model_pca, X_train_tensor_pca, y_train_tensor_pca, X_val_tensor_pca, y_val_tensor_pca)

    dump(mlp_model_pca, 'mlp_model_pca.joblib')

else:

    mlp_model_pca = load('mlp_model_pca.joblib')

In [84]:
input_size_resized = X_train_tensor_pca_resized.shape[1]
hidden_size_resized = 128
output_size_resized = len(torch.unique(y_train_tensor_pca_resized))
mlp_model_pca_resized = MLPClassifier(input_size_resized, hidden_size_resized, output_size_resized)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model_pca_resized.parameters(), lr=0.001)

In [85]:
if not trained_mlp:
    
    train_model_mlp(mlp_model_pca_resized, train_loader_pca_resized, criterion, optimizer, num_epochs=10)

    eval_mlp_model(mlp_model_pca_resized, X_train_tensor_pca_resized, y_train_tensor_pca_resized, X_val_tensor_pca_resized, y_val_tensor_pca_resized)

    dump(mlp_model_pca_resized, 'mlp_model_pca_resized.joblib')

else:

    mlp_model_pca_resized = load('mlp_model_pca_resized.joblib')

#### **Regularized MLP**

In [86]:
# Define the MLP model with regularization
class RegularizedMLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5, weight_decay=1e-5):
        super(RegularizedMLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.weight_decay = weight_decay

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
# Define training function with regularization
def train_model_mlp_reg(model, train_loader, X_val_tensor, y_val_tensor, criterion, optimizer, device, num_epochs=10):

    for epoch in range(num_epochs):

        model.train()
        total_correct = 0
        total_samples = 0

        for inputs, labels in train_loader:

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # L2 regularization
            l2_reg = 0

            for param in model.parameters():
                l2_reg += torch.norm(param)
            loss += model.weight_decay * l2_reg
            loss.backward()
            optimizer.step()
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        
        train_accuracy = total_correct / total_samples
        
        # Evaluate on validation set
        model.eval()
        with torch.no_grad():
            outputs = model(X_val_tensor)
            _, predicted = torch.max(outputs, 1)
            val_accuracy = accuracy_score(y_val_tensor.cpu().numpy(), predicted.cpu().numpy())
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}")

In [87]:
input_size = X_train_tensor_pca.shape[1]
hidden_size = 128
output_size = len(torch.unique(y_train_tensor_pca))
reg_mlp_model_pca = RegularizedMLPClassifier(input_size, hidden_size, output_size).to(device)
# Set up criterion, optimizer, and other parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(reg_mlp_model_pca.parameters(), lr=0.001)
num_epochs = 10

In [88]:
if not trained_mlp:
    # Train the model with regularization
    train_model_mlp_reg(reg_mlp_model_pca, train_loader_pca, X_val_tensor_pca, y_val_tensor_pca, criterion, optimizer, device, num_epochs)

In [89]:
input_size_resized = X_train_tensor_pca_resized.shape[1]
hidden_size_resized = 128
output_size_resized = len(torch.unique(y_train_tensor_pca_resized))
reg_mlp_model_pca_resized = RegularizedMLPClassifier(input_size_resized, hidden_size_resized, output_size_resized).to(device)
# Set up criterion, optimizer, and other parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(reg_mlp_model_pca_resized.parameters(), lr=0.001)
num_epochs = 10

In [90]:
if not trained_mlp:
    train_model_mlp_reg(reg_mlp_model_pca_resized, train_loader_pca_resized, X_val_tensor_pca_resized, y_val_tensor_pca_resized, criterion, optimizer, device, num_epochs)

#### **CNN**

In [91]:
# Define CNN architecture
class CNN(nn.Module):
    def __init__(self, num_classes, c, h, w):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(c, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.relu = nn.ReLU()
        # Calculate input size to the fully connected layer dynamically
        self.fc_input_size = self._calculate_conv_output_size(c, h, w)

        self.fc1 = nn.Linear(self.fc_input_size, 512)
        self.fc2 = nn.Linear(512, num_classes)


    def _calculate_conv_output_size(self, c, h, w):
        # Dummy input to calculate the output size of the convolutional layers
        x = torch.randn(1, c, h, w)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        return x.size(1) * x.size(2) * x.size(3)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model_cnn(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10, patience=5, weight_decay=1e-4):
    model.to(device)

    best_val_loss = float('inf')
    no_improvement = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Apply L2 regularization (weight decay)
            l2_reg = torch.tensor(0., device=device)
            for param in model.parameters():
                l2_reg += torch.norm(param)**2
            loss += weight_decay * l2_reg

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)

            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / total_train

        # Validation
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

            val_accuracy = correct_val / total_val
            val_loss /= len(val_loader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            no_improvement = 0
        else:
            no_improvement += 1
            if no_improvement >= patience:
                print(f'Early stopping after {epoch+1} epochs.')
                break

In [92]:
cnn_model_125x94 = CNN(num_classes= len(np.unique(y_train_resampled_smote)), c=3, h=125, w=94).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model_125x94.parameters(), lr=0.001)

In [93]:
if not trained_cnn_125x94:

    train_model_cnn(cnn_model_125x94, train_loader_3d, val_loader_3d, criterion, optimizer, device, num_epochs=25)
    dump(cnn_model_125x94, 'cnn_model_125x94.joblib')

else:
    cnn_model_125x94 = load('cnn_model_125x94.joblib')

In [94]:
cnn_model_resized = CNN(num_classes= len(np.unique(y_train_resampled_smote_resized)), c=3, h=32, w=32).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model_resized.parameters(), lr=0.001)

In [95]:
if not trained_cnn_resized:

    train_model_cnn(cnn_model_resized, train_loader_3d_resized, val_loader_3d_resized, criterion, optimizer, device, num_epochs=25, patience=10)
    # dump(cnn_model_resized, 'cnn_model_resized.joblib')

else:
    pass
    # cnn_model_resized = load('cnn_model_resized.joblib')

#### **HOG**

In [192]:
class HOGFeatureExtractor:
    def __init__(self, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
    
    def extract_features(self, images):

        hog_features = []
        hog_images = []

        for image in images:

            # Compute HOG features and visualization
            features, hog_image = hog(image, orientations=self.orientations, 
                                      pixels_per_cell=self.pixels_per_cell,
                                      cells_per_block=self.cells_per_block, 
                                      visualize=True, transform_sqrt=True)
            hog_features.append(features)
            hog_images.append(hog_image)
        return np.array(hog_features), np.array(hog_images)

    def prepare_data(self, data):
        features, hog_images = self.extract_features(data)
        # Normalize features
        features = np.array(features)
        features = features / np.linalg.norm(features, axis=1)[:, None]
        return features, hog_images

In [172]:
if not extracted_hog:
    
    hog_extractor = HOGFeatureExtractor(orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

    X_train_hog_features, X_train_hog_images = hog_extractor.prepare_data(X_train_gray_resampled_smote_2d)
    X_val_hog_features, X_val_hog_images = hog_extractor.prepare_data(X_val_gray_2d)
    X_test_hog_features, X_test_hog_images = hog_extractor.prepare_data(X_test_gray_2d)

    dump(X_train_hog_features, 'X_train_hog_features.joblib')
    dump(X_val_hog_features, 'X_val_hog_features.joblib')
    dump(X_test_hog_features, 'X_test_hog_features.joblib')


    # hog_extractor_resized = HOGFeatureExtractor()

    # X_train_hog_features_resized, X_train_hog_images_resized = hog_extractor_resized.prepare_data(X_train_resampled_smote_3d_resized)
    # X_val_hog_features_resized, X_val_hog_images_resized = hog_extractor_resized.prepare_data(X_val_3d_resized)
    # X_test_hog_features_resized, X_test_hog_images_resized = hog_extractor_resized.prepare_data(X_test_3d_resized)

    # dump(X_train_hog_features_resized, 'X_train_hog_features_resized.joblib')
    # dump(X_val_hog_features_resized, 'X_val_hog_features_resized.joblib')
    # dump(X_test_hog_features_resized, 'X_test_hog_features_resized.joblib')

else:
    X_train_hog_features = load('X_train_hog_features.joblib')
    X_val_hog_features = load('X_val_hog_features.joblib')
    X_test_hog_features = load('X_test_hog_features.joblib')

    X_train_hog_features_resized = load('X_train_hog_features_resized.joblib')
    X_val_hog_features_resized = load('X_val_hog_features_resized.joblib')
    X_test_hog_features_resized = load('X_test_hog_features_resized.joblib')

##### **PCA on hog features**

In [174]:
if not hog_pca_transformed:

    pca_hog = PCA(n_components=100)
    pca_hog_resized = PCA(n_components=100)

    X_train_hog_features_pca = pca_hog.fit_transform(X_train_hog_features)
    X_val_hog_features_pca = pca_hog.transform(X_val_hog_features)
    X_test_hog_features_pca = pca_hog.transform(X_test_hog_features)

    dump(X_train_hog_features_pca, 'X_train_hog_features_pca.joblib')
    dump(X_val_hog_features_pca, 'X_val_hog_features_pca.joblib')
    dump(X_test_hog_features_pca, 'X_test_hog_features_pca.joblib')

    # X_train_hog_features_resized_pca = pca_hog_resized.fit_transform(X_train_hog_features_resized)
    # X_val_hog_features_resized_pca = pca_hog_resized.transform(X_val_hog_features_resized)
    # X_test_hog_features_resized_pca = pca_hog_resized.transform(X_test_hog_features_resized)

    # dump(X_train_hog_features_resized_pca, 'X_train_hog_features_resized_pca.joblib')
    # dump(X_val_hog_features_resized_pca, 'X_val_hog_features_resized_pca.joblib')
    # dump(X_test_hog_features_resized_pca, 'X_test_hog_features_resized_pca.joblib')

else:

    X_train_hog_features_pca = load('X_train_hog_features_pca.joblib')
    X_val_hog_features_pca = load('X_val_hog_features_pca.joblib')
    X_test_hog_features_pca = load('X_test_hog_features_pca.joblib')

    X_train_hog_features_resized_pca = load('X_train_hog_features_resized_pca.joblib')
    X_val_hog_features_resized_pca = load('X_val_hog_features_resized_pca.joblib')
    X_test_hog_features_resized_pca = load('X_test_hog_features_resized_pca.joblib')

##### **SVM on PCA transformed hog features**

In [175]:
if not trained_hog_svm:

    # Define SVM classifier
    svm_hog_pca = SVC()

    # Define parameter grid for grid search
    svm_param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10]} 

    # Perform grid search for SVM on PCA transformed hog features
    svm_grid_search_pca = GridSearchCV(svm_hog_pca, svm_param_grid, cv=5)
    svm_grid_search_pca.fit(X_train_hog_features_pca, y_train_resampled_smote)

    # Get best hyperparameters for SVM
    best_svm_params_hog_pca = svm_grid_search_pca.best_params_

    # Train SVM classifier with best hyperparameters
    best_svm_hog_pca = SVC(**best_svm_params_hog_pca)
    best_svm_hog_pca.fit(X_train_hog_features_pca, y_train_resampled_smote)

    # Save the trained SVM model
    dump(best_svm_hog_pca, 'best_svm_model_hog_pca.joblib')

    # Predictions on train, val, and test sets
    train_pred_svm_hog_pca = best_svm_hog_pca.predict(X_train_hog_features_pca)
    val_pred_svm_hog_pca = best_svm_hog_pca.predict(X_val_hog_features_pca)
    test_pred_svm_hog_pca = best_svm_hog_pca.predict(X_test_hog_features_pca)

    # Calculate accuracies
    train_accuracy_svm_hog_pca = accuracy_score(y_train_resampled_smote, train_pred_svm_hog_pca)
    val_accuracy_svm_hog_pca = accuracy_score(y_val, val_pred_svm_hog_pca)
    test_accuracy_svm_hog_pca = accuracy_score(y_test, test_pred_svm_hog_pca)

    # Print accuracies
    print("SVM(PCA):")
    print(f"Best params(PCA): {best_svm_params_hog_pca}")
    print(f"Train Accuracy(PCA): {train_accuracy_svm_hog_pca}")
    print(f"Validation Accuracy(PCA): {val_accuracy_svm_hog_pca}")
    print(f"Test Accuracy(PCA): {test_accuracy_svm_hog_pca}")

else:
    
    # Load the saved SVM model
    best_svm_hog_pca = load('best_svm_model_hog_pca.joblib')

    # Predictions on train, val, and test sets
    train_pred_svm_hog_pca = best_svm_hog_pca.predict(X_train_hog_features_pca)
    val_pred_svm_hog_pca = best_svm_hog_pca.predict(X_val_hog_features_pca)
    test_pred_svm_hog_pca = best_svm_hog_pca.predict(X_test_hog_features_pca)

    # Calculate accuracies
    train_accuracy_svm_hog_pca = accuracy_score(y_train_resampled_smote, train_pred_svm_hog_pca)
    val_accuracy_svm_hog_pca = accuracy_score(y_val, val_pred_svm_hog_pca)
    test_accuracy_svm_hog_pca = accuracy_score(y_test, test_pred_svm_hog_pca)

    # Print accuracies
    print("SVM(PCA):")
    # print(f"Best params(PCA): {best_svm_params_hog_pca}")
    print(f"Train Accuracy(PCA): {train_accuracy_svm_hog_pca}")
    print(f"Validation Accuracy(PCA): {val_accuracy_svm_hog_pca}")
    print(f"Test Accuracy(PCA): {test_accuracy_svm_hog_pca}")

SVM(PCA):
Best params(PCA): {'C': 10, 'kernel': 'rbf'}
Train Accuracy(PCA): 1.0
Validation Accuracy(PCA): 0.7349397590361446
Test Accuracy(PCA): 0.7369477911646586


#### **LBP**

In [186]:
class LBPFeatureExtractor:
    def __init__(self, radius=1, n_points=8, method='default'):
        self.radius = radius
        self.n_points = n_points
        self.method = method

    def extract_features(self, images):
        
        lbp_features = []
        lbp_images = []

        for image in images:

            # Compute LBP features and visualization
            lbp_image = local_binary_pattern(image, self.n_points, self.radius, method=self.method)
            lbp_features.append(lbp_image)
            lbp_images.append(lbp_image)

        return np.array(lbp_features), np.array(lbp_images)

    def prepare_data(self, data):
        features, lbp_images = self.extract_features(data)
        # Flatten LBP images to make them suitable for model input
        features = features.reshape(features.shape[0], -1)
        return features, lbp_images

In [189]:
if not extracted_lbp:

    lbp_extractor = LBPFeatureExtractor()

    X_train_lbp_features, X_train_lbp_images = lbp_extractor.prepare_data(X_train_gray_resampled_smote_2d)

    X_val_lbp_features, X_val_lbp_images = lbp_extractor.prepare_data(X_val_gray_2d)

    X_test_lbp_features, X_test_lbp_images = lbp_extractor.prepare_data(X_test_gray_2d)

    dump(X_train_lbp_features, 'X_train_lbp_features.joblib')
    dump(X_val_lbp_features, 'X_val_lbp_features.joblib')
    dump(X_test_lbp_features, 'X_test_lbp_features.joblib')

else:

    X_train_lbp_features = load('X_train_lbp_features.joblib')
    X_val_lbp_features = load('X_val_lbp_features.joblib')
    X_test_lbp_features = load('X_test_lbp_features.joblib')

c:\Users\91883\AppData\Local\Programs\Python\Python311\Lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


##### **PCA on LBP extracted features**

In [193]:
if not lbp_pca_transformed:

    pca_lbp = PCA(n_components=100)

    X_train_lbp_features_pca = pca_lbp.fit_transform(X_train_hog_features)
    X_val_lbp_features_pca = pca_lbp.transform(X_val_hog_features)
    X_test_lbp_features_pca = pca_lbp.transform(X_test_hog_features)

    dump(X_train_lbp_features_pca, 'X_train_lbp_features_pca.joblib')
    dump(X_val_lbp_features_pca, 'X_val_lbp_features_pca.joblib')
    dump(X_test_lbp_features_pca, 'X_test_lbp_features_pca.joblib')

else:

    X_train_lbp_features_pca = load('X_train_lbp_features_pca.joblib')
    X_val_lbp_features_pca = load('X_val_lbp_features_pca.joblib')
    X_test_lbp_features_pca = load('X_test_lbp_features_pca.joblib')

##### **SVM on PCA transformed LBP features**

In [196]:
if not trained_lbp_svm:

    # Define SVM classifier
    svm_lbp_pca = SVC()

    # Define parameter grid for grid search
    svm_param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10]} 

    # Perform grid search for SVM on PCA transformed hog features
    svm_grid_search_pca = GridSearchCV(svm_lbp_pca, svm_param_grid, cv=5)
    svm_grid_search_pca.fit(X_train_lbp_features_pca, y_train_resampled_smote)

    # Get best hyperparameters for SVM
    best_svm_params_lbp_pca = svm_grid_search_pca.best_params_

    # Train SVM classifier with best hyperparameters
    best_svm_lbp_pca = SVC(**best_svm_params_lbp_pca)
    best_svm_lbp_pca.fit(X_train_lbp_features_pca, y_train_resampled_smote)

    # Save the trained SVM model
    dump(best_svm_lbp_pca, 'best_svm_model_hog_pca.joblib')

    # Predictions on train, val, and test sets
    train_pred_svm_lbp_pca = best_svm_lbp_pca.predict(X_train_lbp_features_pca)
    val_pred_svm_lbp_pca = best_svm_lbp_pca.predict(X_val_lbp_features_pca)
    test_pred_svm_lbp_pca = best_svm_lbp_pca.predict(X_test_lbp_features_pca)

    # Calculate accuracies
    train_accuracy_svm_lbp_pca = accuracy_score(y_train_resampled_smote, train_pred_svm_lbp_pca)
    val_accuracy_svm_lbp_pca = accuracy_score(y_val, val_pred_svm_lbp_pca)
    test_accuracy_svm_lbp_pca = accuracy_score(y_test, test_pred_svm_lbp_pca)

    # Print accuracies
    print("SVM(PCA):")
    print(f"Best params(PCA): {best_svm_params_lbp_pca}")
    print(f"Train Accuracy(PCA): {train_accuracy_svm_lbp_pca}")
    print(f"Validation Accuracy(PCA): {val_accuracy_svm_lbp_pca}")
    print(f"Test Accuracy(PCA): {test_accuracy_svm_lbp_pca}")

else:
    
    # Load the saved SVM model
    best_svm_lbp_pca = load('best_svm_model_hog_pca.joblib')

    # Predictions on train, val, and test sets
    train_pred_svm_lbp_pca = best_svm_lbp_pca.predict(X_train_lbp_features_pca)
    val_pred_svm_lbp_pca = best_svm_lbp_pca.predict(X_val_lbp_features_pca)
    test_pred_svm_lbp_pca = best_svm_lbp_pca.predict(X_test_lbp_features_pca)

    # Calculate accuracies
    train_accuracy_svm_lbp_pca = accuracy_score(y_train_resampled_smote, train_pred_svm_lbp_pca)
    val_accuracy_svm_lbp_pca = accuracy_score(y_val, val_pred_svm_lbp_pca)
    test_accuracy_svm_lbp_pca = accuracy_score(y_test, test_pred_svm_lbp_pca)

    # Print accuracies
    print("SVM(PCA):")
    # print(f"Best params(PCA): {best_svm_params_lbp_pca}")
    print(f"Train Accuracy(PCA): {train_accuracy_svm_lbp_pca}")
    print(f"Validation Accuracy(PCA): {val_accuracy_svm_lbp_pca}")
    print(f"Test Accuracy(PCA): {test_accuracy_svm_lbp_pca}")

SVM(PCA):
Best params(PCA): {'C': 10, 'kernel': 'rbf'}
Train Accuracy(PCA): 1.0
Validation Accuracy(PCA): 0.7349397590361446
Test Accuracy(PCA): 0.7309236947791165
